### Домашнее задание 4

В качестве домашнего задания Вам необходимо
Ответьте на вопросы:
- В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?
- Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы
- Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:
1) Какой датасет используют авторы?
2) Что используют в качестве признаков?
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко


##### Практическая часть. Проверим подключение модуля с метриками и предобработкой. Проверка установки lightfm

In [1]:
!pip install lightfm

In [37]:
import warnings
warnings.simplefilter("ignore")

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [39]:
import lightfm.evaluation
pr = lightfm.evaluation.precision_at_k

In [40]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [41]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [42]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [46]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


##### Lightfm установлен, модули подключены

### Теоретическая часть

##### В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?

Коллаборативная фильтрация использует предсказания на основе историй взаимодействий user-item, схожести взаимодействий различных пользователей. Гибрадная система в использует колаборативную фильтрацию + контентную фильтрацию. а так же использует матрицу весов для предсказаний. Благодаря такому подходу можно применять гибридную систему при "холодном старте" и при разраженной матрице user-item

##### Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы

Рекомендации фильмов в видеосервисе (Netflix и т.п.)
Рекомендации товаров в маркетплейсах (Amazon и т.п.)

##### Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/ Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

В модели hh, помимо использованных в наших моделях метрик (ndcg), используется time-based валидация. То есть, на сколько я понял, происходит сравнение уже рекомендованного моделью с тем, что произошло после взаимодействия пользователя с поиском. Так же в их моделях до рекомендации применены линейные алгоритмы и XGboost для разделения запросов/резюме/вакансий, а так же для их ранжирования.

Про проблемы затрудняюсь ответить.

##### На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы: 1) Какой датасет используют авторы? 2) Что используют в качестве признаков? 3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко

1. Первый датасет - MovieLens (рейтинги фильмов) в комбинации с датасетом Tag Genome (описания фильмов). Low-sparsity, т.е. низкоразряженный датасет. Второй датасет - CrossValidated, так же имеющий дополнительное описанию ответивших ('About Me')
2. Использовались 3 варианта. Первый - используются tags. Второй - tags + ids (ids - идентификатор итемов). Третий - tags + about (about был доступен только для второго датасета). tags и ids - признаки, about - словарь, который был разделен на слова и был токенизирован.
3. MF - matrix factorization (то есть коллаборативная модель, основанная на матрице users-items). LSI-LR - основана на факторизации матрицы item-features. Представляет item-feauters ввиде линейной комбинации элементов, разделяет их на пространство по "категориям", а затем с помощью ленийной регрессии подбирает подходящему юзеру "категорию". LSI-UP - гибридная модель. представляет юзера как комбинацию item векторов. затем к результирующей матрице применяется алгоритм LSI (latent semantic indexing) для получения "скрытых" представлений пользователя. Затем создается таблица в строках которой, юзеры, представленные суммой векторов из скрытых представлений. В столбцах items с которыми было положительное взаимодействие. Скрытые вектора итемов получаются из проекции веторов юзеров на features. предсказание делается на основе скрытых векторов юзеров и итемов.

 